In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

# Carregar o tokenizer e o modelo BERT
tokenizer = BertTokenizer.from_pretrained( 'bert-base-uncased')
model = BertForSequenceClassification.from_pretrained( 'bert-base-uncased')

# Entrada de exemplo
input_text = "Chatbots são muito úteis para automação."

# Tokenização
input_ids = tokenizer(input_text, return_tensors='pt')

# Predição
outputs = model(**input_ids)
probs = softmax(outputs.logits, dim=-1)

print("Probabilidades de classe:", probs)

In [ ]:
from transformers import BertTokenizer, BertModel

# Carregar o tokenizer e o modelo BERT pré-treinado
tokenizer = BertTokenizer.from_pretrained( 'bert-base-uncased')
model = BertModel.from_pretrained( 'bert-base-uncased')

# Exemplo de texto
texto = "Machine learning is fascinating."

# Tokenização
input_ids = tokenizer(texto, return_tensors='pt')['input_ids']

# Obtenção das representações do modelo BERT
outputs = model(input_ids)
last_hidden_states = outputs.last_hidden_state

print("Representações BERT:", last_hidden_states)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Carregar o tokenizer e o modelo GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Entrada de exemplo
input_text = "Chatbots modernos podem"

# Tokenização
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Geração de texto
outputs = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decodificação e exibição do texto gerado
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Texto gerado:", generated_text)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Carregar o tokenizer e o modelo GPT-2 pré-treinado
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Exemplo de texto
input_text = "In the future, artificial intelligence will"

# Tokenização
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Geração de texto
outputs = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decodificação e exibição do texto gerado
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Texto gerado:", generated_text)

In [ ]:
# pip install transformers[torch]
# pip install sentencepiece

from transformers import T5Tokenizer, T5ForConditionalGeneration

# Carregar o tokenizer e o modelo T5
tokenizer = T5Tokenizer.from_pretrained( 't5-small')
model = T5ForConditionalGeneration.from_pretrained( 't5-small')

# Entrada de exemplo
input_text = "translate English to German: The weather is nice today."

# Tokenização
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Geração de texto
outputs = model.generate(input_ids)

# Decodificação e exibição do texto gerado
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Tradução gerada:", generated_text)

In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel

# Carregar o tokenizer e o modelo XLNet
tokenizer = XLNetTokenizer.from_pretrained( 'xlnet-base-cased')
model = XLNetLMHeadModel.from_pretrained( 'xlnet-base-cased')

# Entrada de exemplo
input_text = "Natural Language Processing is"

# Tokenização
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Geração de texto
outputs = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decodificação e exibição do texto gerado
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Texto gerado com XLNet:", generated_text)

In [ ]:
# pip install datasets
# pip install transformers[torch]

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

import os
os.environ["WANDB_DISABLED"] = "true"

# Carregar o dataset IMDb
dataset = load_dataset("imdb")

# Carregar o tokenizer e o modelo BERT
tokenizer = BertTokenizer.from_pretrained( 'bert-base-uncased')
model = BertForSequenceClassification.from_pretrained( 'bert-base-uncased')

# Tokenizar os dados
def tokenize_function(examples):
return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Definir argumentos de treinamento
training_args = TrainingArguments(
	output_dir='./results',
	learning_rate=2e-5,
	report_to="none",
	per_device_train_batch_size=16,
	per_device_eval_batch_size=16,
	num_train_epochs=3,
	weight_decay=0.01,
)

# Criar o Trainer
trainer = Trainer(
model=model,
args=training_args,
train_dataset=tokenized_datasets['train'],
eval_dataset=tokenized_datasets['test'],
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
eval_result = trainer.evaluate()
print(f"Resultado da Avaliação: {eval_result}")

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import BertTokenizer, BertModel
import torch

# Carregar o tokenizer e o modelo para as consultas (questions)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(
"facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained(
"facebook/dpr-question_encoder-single-nq-base")

# Carregar o tokenizer e o modelo para os contextos (passages)
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained( "facebook/dpr-ctx_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained( "facebook/dpr-ctx_encoder-single-nq-base")

# Exemplo de consulta
query = "What is Retrieval-Augmented Generation?"

# Codificar a consulta
query_input = question_tokenizer(query, return_tensors="pt")
query_embedding = question_encoder(**query_input).pooler_output

# Exemplo de passagens
passages = [
    "Retrieval-Augmented Generation is a technique that combines retrieval of relevant information with text generation.",
    "It allows for more accurate and contextually relevant answers by consulting external knowledge bases.",
    "RAG is particularly useful in scenarios where the information required to answer a query is not present in the training data of the language model."
]

# Codificar as passagens
passage_inputs = context_tokenizer(passages, padding=True, truncation=True, return_tensors="pt")
passage_embeddings = context_encoder(
    **passage_inputs).pooler_output

# Calcular similaridade e selecionar a passagem mais relevante
similarity_scores = torch.matmul(query_embedding, passage_embeddings.T)
best_passage_index = torch.argmax(similarity_scores, dim=1).item()
best_passage = passages[best_passage_index]

print("Passagem mais relevante:", best_passage)

In [ ]:
# pip install transformers[torch]
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import BertTokenizer, BertModel
import torch

# Carregar o tokenizer e o modelo para as consultas (questions)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(
"facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained(
"facebook/dpr-question_encoder-single-nq-base")

# Carregar o tokenizer e o modelo para os contextos (passages)
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained( "facebook/dpr-ctx_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained( "facebook/dpr-ctx_encoder-single-nq-base")

# Exemplo de consulta
query = "What is Retrieval-Augmented Generation?"

# Codificar a consulta
query_input = question_tokenizer(query, return_tensors="pt")
query_embedding = question_encoder(**query_input).pooler_output

# Exemplo de passagens
passages = [
    "Retrieval-Augmented Generation is a technique that combines retrieval of relevant information with text generation.",
    "It allows for more accurate and contextually relevant answers by consulting external knowledge bases.",
    "RAG is particularly useful in scenarios where the information required to answer a query is not present in the training data of the language model."
]

# Codificar as passagens
passage_inputs = context_tokenizer(passages, padding=True, truncation=True, return_tensors="pt")
passage_embeddings = context_encoder
   **passage_inputs).pooler_output

# Calcular similaridade e selecionar a passagem mais relevante
similarity_scores = torch.matmul(query_embedding, passage_embeddings.T)
best_passage_index = torch.argmax(similarity_scores, dim=1).item()
best_passage = passages[best_passage_index]

# Carregar o tokenizer e o modelo BART
bart_tokenizer = BartTokenizer.from_pretrained(
"facebook/bart-large")
bart_model = BartForConditionalGeneration.from_pretrained(
"facebook/bart-large")

# Concatenar a consulta com a passagem relevante
input_text = query + " " + best_passage

# Codificar e gerar resposta
input_ids = bart_tokenizer.encode(input_text, return_tensors="pt")
generated_ids = bart_model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
generated_text = bart_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Resposta gerada:", generated_text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from google.colab import userdata

from google.colab import userdata
userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token=userdata.get('HF_TOKEN'))

# Carregar o tokenizer e o modelo LLaMA
tokenizer = AutoTokenizer.from_pretrained(
"meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained(
"meta-llama/Meta-Llama-3-8B")

# Texto de entrada
input_text = "Artificial intelligence is transforming the world of"

# Tokenizar e gerar texto
input_ids = tokenizer.encode(input_text, return_tensors="pt")
generated_ids = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Texto gerado:", generated_text)
\end{lstlisting} 

In [ ]:
from transformers import pipeline
pipe = pipeline('text-generation', model='gpt2')
input = 'Olá, como vai você?'
output = pipe(input)
print(output)

In [ ]:
# pip install transformers[torch]
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
input= "Olá, como vai você?"	
token_id = tokenizer(input)
print(token_id)

In [ ]:
# pip install langchain

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

model = init_chat_model("gpt-4o-mini", 
	model_provider="openai", 						openai_api_key=OPENAI_API_KEY)

user_message = HumanMessage(content="Quem é o presidente do Brasil?")

response = model.invoke([user_message])

print(response.content)

In [ ]:
# para instalação acesse https://github.com/Mangaba-ai/mangaba_ai
from mangaba import Team, Agent

# Criar uma equipe de agentes
monitor = Team("Autorregulação")

# Adicionar agentes especializados
planejador = Agent("Plano de Estudo", role="Gera Plano de Estudo com cronograma baseado no tempo disponível do estudante")
apoio = Agent("Suporte Emocional", role="Motiva o estudante")
pesquisador = Agent("Guia de Estudo", role="Busca materiais didáticos para o estudante")

# Adicionar agentes à equipe
monitor.add_agents([pesquisador, apoio, planejador])

# Definir uma tarefa complexa
result = monitor.solve(
  "Pesquise sobre Redes Neurais Artificiais"
)

print(result.output)

In [ ]:
Hugging Face Transformers.}]
from transformers import AutoTokenizer, AutoModelForCausalLM, DPRQuestionEncoder, DPRContextEncoder

# Carregar o tokenizer e o modelo LLaMA
llama_tokenizer = AutoTokenizer.from_pretrained(
"meta-llama/LLaMA-7B")
llama_model = AutoModelForCausalLM.from_pretrained(
"meta-llama/LLaMA-7B")

# Configurar DPR para recuperação de passagens
question_encoder = DPRQuestionEncoder.from_pretrained( "facebook/dpr-question_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained( "facebook/dpr-ctx_encoder-single-nq-base")

In [ ]:
# Este código deve ser executado na sequencia, depois do código anterior.
import torch

def retrieve_relevant_passage(query, passages):
	query_input = question_tokenizer(query, return_tensors="pt")
	query_embedding = question_encoder(
       **query_input).pooler_output

	passage_inputs = context_tokenizer(passages, padding=True, truncation=True, return_tensors="pt")
	passage_embeddings = context_encoder(
      **passage_inputs).pooler_output

	similarity_scores = torch.matmul(query_embedding, passage_embeddings.T)
	best_passage_index = torch.argmax(similarity_scores, dim=1).item()

	return passages[best_passage_index]

# Exemplo de passagens
passages = [
	"LLaMA é um modelo de linguagem desenvolvido pela Meta AI.",
	"RAG combina recuperação de informações com geração de texto.",
	"GPT-3 é um dos maiores modelos de linguagem disponíveis."
]

# Entrada do usuário
user_input = "O que é LLaMA?"

# Recuperar a passagem mais relevante
relevant_passage = retrieve_relevant_passage(user_input, passages)

print(f"Passagem mais relevante: {relevant_passage}")

In [ ]:
# Concatenar a consulta com a passagem relevante
input_text = user_input + " " + relevant_passage

# Geração da resposta
input_ids = llama_tokenizer.encode(input_text, return_tensors="pt")
generated_ids = llama_model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
response = llama_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Resposta do chatbot:", response)